### Import library

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle

In [14]:
batch_size = 128
block_size = 64
max_iters = 5000
eval_iters = 100
n_embd = 384
n_layer = 4
n_head = 4
learning_rate = 3e-4
dropout = 0.2

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
with open('data/truyenkieu.txt', 'r', encoding = 'utf-8') as f:
    data = f.read()
print(data[:150])

Phần 1:
Trăm năm trong cõi người ta,
Chữ tài chữ mệnh khéo là ghét nhau.
Trải qua một cuộc bể dâu,
Những điều trông thấy mà đau đớn lòng.
Lạ gì bỉ sắc


In [5]:
vocab = sorted(set(data))
vocab_size = len(vocab)
print(vocab_size, vocab)

128 ['\n', ' ', '!', ',', '-', '.', '1', '2', '3', '4', ':', '?', 'A', 'B', 'C', 'D', 'E', 'G', 'H', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'Â', 'Ê', 'Ô', 'à', 'á', 'â', 'ã', 'è', 'é', 'ê', 'ì', 'í', 'ò', 'ó', 'ô', 'õ', 'ù', 'ú', 'ý', 'Ă', 'ă', 'Đ', 'đ', 'ĩ', 'ũ', 'Ơ', 'ơ', 'Ư', 'ư', 'ạ', 'ả', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ẹ', 'ẻ', 'ẽ', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ỉ', 'ị', 'ọ', 'ỏ', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'ụ', 'ủ', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'ỳ', 'ỷ', 'ỹ']


### Encoder and Decoder

In [6]:
string_to_int = { ch:i for i, ch in enumerate(vocab)}
int_to_string = { i:ch for i, ch in enumerate(vocab)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda i: ''.join(int_to_string[c] for c in i)

data = torch.tensor(encode(data), dtype=torch.long)
print(decode(encode('hello')))

hello


### Get batch

In [7]:
pos = int(0.9 * len(data))
train_split = data[:pos]
val_split = data[pos:]

def get_batch(split):
    data_set = train_split if split == 'train' else val_split
    num = torch.randint(len(data_set) - block_size, (batch_size,))
    x = torch.stack([data_set[i: i + block_size] for i in num])
    y = torch.stack([data_set[i + 1: i + block_size + 1] for i in num])
    x, y = x.to(device), y.to(device)
    return x, y

### Your model here

In [16]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        B, T, C = x.shape
        
        k = self.key(x) # B, T, Hs
        q = self.query(x) # B, T, Hs

        dot_prod = k @ q.transpose(-2, -1) # --> B, T, T
        wei = dot_prod * k.shape[-1] ** -0.5 # --> B, T, T
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out 
        


class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd * 4),
            nn.ReLU(),
            nn.Linear(n_embd * 4,n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_head * head_size , n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out 

class Decoder_Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ffwd = FeedForward(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
        


class TruyenKieuLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.decoder_blocks = nn.Sequential(*[Decoder_Block(n_embd, n_head = n_head) for i in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    

    def forward(self, index, targets = None):
        B, T = index.shape
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.decoder_blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index_cond)
            
            logits = logits[:, -1,:]
            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next) , dim = 1)
            
        return index
model = TruyenKieuLanguageModel(vocab_size)
# print('loading model parameters...')
# with open('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
# print('loaded successfully!')
m = model.to(device)

### Loss data

In [17]:
@torch.no_grad
def loss_detail():
    out = {}
    model.eval()
    for split in ['train', 'validate']:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[i] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Training Loop

In [18]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello! Can you see me??ổặDẵẫDâẻBểìETkàâẻKsàí1YũQỉđpsỷqPằlxlùưéeéổ

òãOtễdHTsằyƯ:ứnƠởủÂeÊễứĐípồgo-Ướẵ-í1íểữộdởlỉ1ữủễơOOcdMỷ


In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
for iter in range (max_iters):
    if iter % eval_iters == 0:
        losses = loss_detail()
        print(f'Vòng huấn luyện: {iter}, loss: {losses}')
    elif iter % 50 == 0:
        print(f'Vòng huấn luyện: {iter}')
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

Vòng huấn luyện: 0, loss: {'train': tensor(4.9265), 'validate': tensor(4.9303)}
Vòng huấn luyện: 50
Vòng huấn luyện: 100, loss: {'train': tensor(2.0804), 'validate': tensor(2.0988)}
Vòng huấn luyện: 150
Vòng huấn luyện: 200, loss: {'train': tensor(1.8399), 'validate': tensor(1.8692)}
Vòng huấn luyện: 250
Vòng huấn luyện: 300, loss: {'train': tensor(1.7218), 'validate': tensor(1.7516)}
Vòng huấn luyện: 350
Vòng huấn luyện: 400, loss: {'train': tensor(1.6414), 'validate': tensor(1.6856)}
Vòng huấn luyện: 450
Vòng huấn luyện: 500, loss: {'train': tensor(1.5899), 'validate': tensor(1.6545)}
Vòng huấn luyện: 550
Vòng huấn luyện: 600, loss: {'train': tensor(1.5381), 'validate': tensor(1.6257)}
Vòng huấn luyện: 650
Vòng huấn luyện: 700, loss: {'train': tensor(1.4897), 'validate': tensor(1.6118)}
Vòng huấn luyện: 750
Vòng huấn luyện: 800, loss: {'train': tensor(1.4441), 'validate': tensor(1.6065)}
Vòng huấn luyện: 850
Vòng huấn luyện: 900, loss: {'train': tensor(1.3899), 'validate': tensor(1.5

In [22]:
import pickle
with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

model saved


In [25]:
prompt = 'Thúy Kiều là ai'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=1000)[0].tolist())
print(generated_chars)

Thúy Kiều là ai thị tì tao!
Rồi ra trở mặt tức thì,
Bớt lời liệu chớ sân si thiệt đời!
Nàng rằng: Thề thốt nặng lời,
Có đâu mà chẳng dám ngăn rời chim xanh.
Đã cho vào bậc bố bề lửa dong,
Rước mời vào thằng bán tơ!
Một nhà đạp nận khan gâm.
Cớ nhà hương mở mây tạnh thong dong,
Thang lan rủ bức hồng trưa,
Đóng thuê thoắt đã đổi thay mang lấy sắc tài e châm.
Vì ta rong theo tiết gọng mái tường,
Dàu dàu ngọn cỏ nửa vàng nửa xanh.
Rằng: Sao trong tiết Thanh minh,
Mà đây hương tuốt thiên tầm như gia.
Được rày như nhọ Vương người dở dang.
Thề hoa chưa ráù thẹn phím nàng ngài kíp vu quan.
Vộc lời chi vui chén thong dong,
Nỗi lòng ai ở trong lòng mà ra tra.
Lại càng ủ dột tỉnh lúc mê,
Máu theo nước mắt hoa lòng trên duy.
Sóng song the với càng,
Nghĩ dày càng tỏ trang tự tình.
Gót đầu mới trướng hồng vững lời đâu?
Sân Lai càng như dại nhớ ra,
Chiều duyên chưa dứt thì bằn chị trao.
Hoa khôi lỡ một lầm hồng xuống môn.
Nghĩ người thôi lại nghĩ mình,
Cảm lòa cho mặt chữ tình cho nên!
Trên mui lăng 